In [5]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pandas_ml_quant
import pandas_quant_data_provider as qd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
print(pandas_ml_quant.__version__)
df = qd.fetch("SPY")[-500:]

0.2.0


In [7]:
with df.model() as m:
    from sklearn.neural_network import MLPClassifier

    from pandas_ml_common import random_splitter
    from pandas_ml_common.sampling.cross_validation import KEquallyWeightEvents
    from pandas_ml_utils import SkModel, PostProcessedFeaturesAndLabels, ClassificationSummary, FittingParameter

    fit = m.fit(
        SkModel(
            MLPClassifier(activation='tanh', hidden_layer_sizes=(60, 50), random_state=42, warm_start=True, max_iter=20),
            PostProcessedFeaturesAndLabels(
                features=
                    [
                        lambda df: df["Close"].ta.macd().ML[['macd.*', 'signal.*']],
                        lambda df: df.ta.adx().ML[['+DI', '-DM', '+DM']],
                        lambda df: df["Close"].ta.mom(),
                        lambda df: df["Close"].ta.apo(),
                        lambda df: df.ta.atr(),
                        lambda df: df["Close"].ta.trix(),
                    ],
                feature_post_processor=lambda df: df.ta.rnn(280),
                labels=[
                    lambda df: df["Close"].ta.future_bband_quantile(include_mean=False).ta.one_hot_encode_discrete()
                ],
                targets=[
                    lambda df: df["Close"].ta.bbands()[["lower", "upper"]]
                ]
            ),
            summary_provider=ClassificationSummary,
        ),
        FittingParameter(
            splitter=random_splitter(test_size=0.4, seed=42),
            cross_validation=KEquallyWeightEvents(n_splits=3),
            epochs=1
        )
    )

fit

exc_type: <class 'ValueError'>
exc_value: ("You might need to pass 'classes' argument for partial fitting", ValueError('classes must be passed on the first call to partial_fit.'))
exc_traceback: <traceback object at 0x7fa93ce61ac0>


ValueError: ("You might need to pass 'classes' argument for partial fitting", ValueError('classes must be passed on the first call to partial_fit.'))

In [ ]:
fit.model._history